# Setup

This notebook is inteded to run on colab on preprocessed images.

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
# !pip install efficientnet_pytorch
!pip install git+https://github.com/rwightman/pytorch-image-models
!pip install pytorch-metric-learning
!pip install faiss-gpu
!pip install imgaug -U
!pip install albumentations -U

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
import math

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm
import scipy

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam
from pytorch_metric_learning import miners, losses, samplers , distances, regularizers 

# Global

In [ ]:
!mkdir /home/data
!unzip -qq /gdrive/MyDrive/Projects/Hotel-ID/data/train-256x256.zip -d /home/data/

In [ ]:
SEED = 42
PROJECT_FOLDER = "/gdrive/My Drive/Projects/Hotel-ID/"
DATA_FOLDER = "/home/data/"

In [ ]:
print(os.listdir(PROJECT_FOLDER))
print(os.listdir(PROJECT_FOLDER + "data"))
print(len(os.listdir(DATA_FOLDER)))

# Helper functions - seed and metric calculator

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
IMG_SIZE = 256

import albumentations as A
import albumentations.pytorch as APT
import cv2 

train_transform = A.Compose([
    # A.Resize(IMG_SIZE, IMG_SIZE),
    # A.CLAHE(p=1), 
    
    A.HorizontalFlip(p=0.75),
    A.VerticalFlip(p=0.25),
    A.ShiftScaleRotate(p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.OpticalDistortion(p=0.25),
    A.IAAPerspective(p=0.25),
    A.CoarseDropout(p=0.5),

    A.RandomBrightness(p=0.75),
    A.ToFloat(),
    APT.transforms.ToTensor(),
])


val_transform = A.Compose([
    # A.Resize(IMG_SIZE, IMG_SIZE),
    # A.CLAHE(p=1),
    A.ToFloat(),
    APT.transforms.ToTensor(),
])

In [ ]:
class HotelTrainDataset:
    def __init__(self, data, transform=None, data_path="train_images/"):
        self.data = data
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_path + record["image"]
        image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
        
        return {
            "image" : transformed["image"],
            "target" : record['hotel_id_code'],
        }

# Model

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_classes=100, embed_size=64, backbone_name="efficientnet_b0"):
        super(EmbeddingNet, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
        )

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.embed_size),
            nn.Dropout(0.2),
            nn.Linear(self.embed_size, n_classes),
        )
        
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        return x

# Model helper functions

In [ ]:
def get_embeds(loader, model, bar_desc="Generating embeds"):
    targets_all = []
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            target = sample['target'].to(args.device)
            output = model(input)
            
            targets_all.extend(target.cpu().numpy())
            outputs_all.extend(output.detach().cpu().numpy())
            
    return targets_all, outputs_all

In [ ]:
def save_checkpoint(model, scheduler, optimizer, epoch, name):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  }

    torch.save(checkpoint, f"{PROJECT_FOLDER}output/checkpoint-{name}.pt")


def load_checkpoint(model, scheduler, optimizer, name):
    checkpoint = torch.load(f"{PROJECT_FOLDER}output/checkpoint-{name}.pt")

    model.load_state_dict(checkpoint["model"])
    scheduler.load_state_dict(checkpoint["scheduler"])
    # do not load optimizer checkpoint, lookahead might have some gradients so it may cuz memory error
    # optimizer.load_state_dict(checkpoint["optimizer"])

    return model, scheduler, optimizer, checkpoint["epoch"]

In [ ]:
def iterate_loader(loader, epochs):
    """
    Iterates through data loader with fake load (empty data) instead
    of reading the real images to speed up. Dataloader has no state_dict
    so manual iterating is need to get the loader with augmentations to 
    correct state.
    """
    loader.dataset.fake_load = True
    with torch.no_grad():
        for i in range(epochs):
            t = tqdm(loader, desc=f"Iterating loader {i+1}/{epochs}")
            for j, sample in enumerate(t):
                images = sample['image']
                targets = sample['target']

    loader.dataset.fake_load = False

In [ ]:
def train_epoch(args, model, loader, miner, triplet_criterion, class_criterion, optimizer, scheduler, epoch):
    losses = []
    targets_all = []
    outputs_all = []
    
    model.train()
    t = tqdm(loader)
    
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        
        images = sample['image'].to(args.device)
        targets = sample['target'].to(args.device)
        
        embeds, outputs = model.embed_and_classify(images)
        miner_pairs = miner(embeds, targets)
        triplet_loss = triplet_criterion(embeds, targets, miner_pairs)
        class_loss = class_criterion(outputs, targets)
        loss = triplet_loss + class_loss
        
        loss.backward()
        optimizer.step()
        
        if scheduler:
            scheduler.step()
                
        losses.append(loss.item())
        targets_all.extend(targets.cpu().numpy())
        outputs_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())

        score = np.mean(targets_all == np.argmax(outputs_all, axis=1))
        desc = f"Epoch {epoch} - Train loss:{loss:0.4f}, score: {score:0.4f}"
        t.set_description(desc)
        
    return np.mean(losses)


def test_closest_match(base_df, base_embeds, valid_targets, valid_embeds, model, distance_func, closest, n_matches=5):
    distance_matrix = distance_func(torch.Tensor(valid_embeds), torch.Tensor(base_embeds)).numpy()

    preds = []
    for i in range(0, len(valid_embeds)):
        tmp_df = base_df.copy()
        tmp_df["distance"] = distance_matrix[i]
        tmp_df = tmp_df.sort_values(by=["distance", "hotel_id"], ascending=closest).reset_index(drop=True)
        preds.extend([tmp_df["hotel_id_code"].unique()[:n_matches]])

    y = np.repeat([valid_targets], repeats=n_matches, axis=0).T
    preds = np.array(preds)
    acc_top_1 = (preds[:, 0] == valid_targets).mean()
    acc_top_5 = (preds == y).any(axis=1).mean()
    print(f"Accuracy: {acc_top_1:0.4f}, top 5 accuracy: {acc_top_5:0.4f}")
    return preds, distance_matrix


def test(base_loader, valid_loader, model, distance_func, closest):
    base_targets, base_embeds = get_embeds(base_loader, model, "Generating embeds for train")
    valid_targets, valid_embeds = get_embeds(valid_loader, model, "Generating embeds for test")
    val_preds, distance_matrix = test_closest_match(base_loader.dataset.data, base_embeds, valid_targets, valid_embeds, model, distance_func, closest)
    return base_embeds, valid_embeds, base_targets, valid_targets, val_preds, distance_matrix

# Prepare data

In [ ]:
def sample_data(n_hotels, min_images, max_images):
    data_df = pd.read_csv(PROJECT_FOLDER + "data/train.csv", parse_dates=["timestamp"])
    sample_df = data_df.groupby("hotel_id").filter(lambda x: (x["image"].nunique() > min_images) & (x["image"].nunique() < max_images))
    sample_df["hotel_id_code"] = sample_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)
    sample_df = sample_df[sample_df["hotel_id_code"] < n_hotels]

    print(f"Subsample with {len(sample_df.hotel_id.unique())} hotels out of {len(data_df.hotel_id.unique())}" + 
          f" with total {len(sample_df)} images ({len(sample_df) / len(data_df) * 100:0.2f} %)")
    
    return sample_df

In [ ]:
# FOR TESTING DIFFERENT SETTING
# data_df = sample_data(1000, 15, 50)

# FOR FINAL TRAINING
data_df = pd.read_csv(PROJECT_FOLDER + "data/train.csv", parse_dates=["timestamp"])
data_df["hotel_id_code"] = data_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

fig = go.Figure()
fig.add_trace(go.Histogram(x=data_df["hotel_id_code"]))
fig.update_xaxes(type="category")
fig.show()

In [ ]:
def train_and_validate(args, data_df):
    model_name = f"siamese-entropy-model-{args.backbone_name}-{IMG_SIZE}x{IMG_SIZE}-{args.embed_size}embeds-{args.n_classes}hotels"
    print(model_name)

    seed_everything(seed=SEED)

    valid_df = data_df.groupby("hotel_id").sample(args.val_samples, random_state=SEED)
    train_df = data_df[~data_df["image"].isin(valid_df["image"])]

    sampler = samplers.MPerClassSampler(train_df.hotel_id_code, m=3, batch_size=args.batch_size, length_before_new_iter=len(train_df))
    train_dataset = HotelTrainDataset(train_df, train_transform, data_path=DATA_FOLDER)
    train_loader = DataLoader(train_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False, sampler=sampler, drop_last=True)
    base_dataset = HotelTrainDataset(train_df, val_transform, data_path=DATA_FOLDER)
    base_loader = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
    valid_dataset = HotelTrainDataset(valid_df, val_transform, data_path=DATA_FOLDER)
    valid_loader = DataLoader(valid_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

    print(f"Base: {len(base_dataset)}\nValidation: {len(valid_dataset)}")

    model = EmbeddingNet(args.n_classes, args.embed_size, args.backbone_name)
    model = model.to(args.device)

    distance = distances.CosineSimilarity()
    miner = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="all")

    triplet_criterion = losses.TripletMarginLoss(margin=0.2,distance=distance, triplets_per_anchor="all") # Accuracy: 0.7320, top 5 accuracy: 0.8640
    class_criterion = nn.CrossEntropyLoss()
    optimizer = Lookahead(torch.optim.AdamW(model.parameters(), lr=args.lr), k=3)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=args.lr,
                    epochs=args.epochs,
                    steps_per_epoch=len(train_loader),
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )

    for epoch in range(0, args.epochs):
        train_loss = train_epoch(args, model, train_loader, miner, triplet_criterion, class_criterion, optimizer, scheduler, epoch)
        base_embeds, valid_embeds, base_targets, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model, distance, closest=False)
        save_checkpoint(model, scheduler, optimizer, epoch, model_name)

    # output = {"base_embeds": base_embeds,
    #           "valid_embeds": valid_embeds,
    #           "base_targets": base_targets,
    #           "valid_targets": valid_targets,
    #           "val_preds": val_preds,
    #           "distance_matrix": distance_matrix,
    #           "train_df" : train_df,
    #           "valid_df": valid_df,
    #           }

    # torch.save(output, f"{PROJECT_FOLDER}output/output-{model_name}.pt")

# Train and evaluate

In [ ]:
# distance = distances.SNRDistance() # Accuracy: 0.7040, top 5 accuracy: 0.8480
# distance = distances.CosineSimilarity() # Accuracy: 0.7460, top 5 accuracy: 0.8780
# distance = distances.DotProductSimilarity() # Accuracy: 0.7460, top 5 accuracy: 0.8780
# distance = distances.LpDistance() # Accuracy: 0.7380, top 5 accuracy: 0.8660

In [ ]:
# %%time 

# class args:
#     epochs = 6
#     lr = 1e-3
#     batch_size = 33
#     num_workers = 2
#     embed_size = 256
#     val_samples = 2
#     backbone_name = "efficientnet_b1"
#     n_classes = data_df["hotel_id_code"].nunique()
#     device = ('cuda' if torch.cuda.is_available() else 'cpu')

# train_and_validate(args, data_df)

In [ ]:
# %%time 

# class args:
#     epochs = 6
#     lr = 1e-3
#     batch_size = 33
#     num_workers = 2
#     embed_size = 256
#     val_samples = 2
#     backbone_name = "eca_nfnet_l0"
#     n_classes = data_df["hotel_id_code"].nunique()
#     device = ('cuda' if torch.cuda.is_available() else 'cpu')

# train_and_validate(args, data_df)

In [ ]:
# %%time 

# class args:
#     epochs = 6
#     lr = 1e-3
#     batch_size = 33
#     num_workers = 2
#     embed_size = 256
#     val_samples = 2
#     backbone_name = "ecaresnet50d_pruned"
#     n_classes = data_df["hotel_id_code"].nunique()
#     device = ('cuda' if torch.cuda.is_available() else 'cpu')

# train_and_validate(args, data_df)